In [24]:
from utils.config import load_datasets, get_train_dataset_indexes

folder = "../outputs/datasets" # load saved data
# dataset_indexes = get_train_dataset_indexes(folder)
dataset_indexes = ['burn']
datasets = load_datasets(folder=folder, names=dataset_indexes)
